# Using Hypothesis to Generate Random Pandas DataFrames
When I started working with [hypothesis](https://hypothesis.readthedocs.io), I couldn't find any examples that showed how to generate a pandas DataFrame.  The following is my solution.  It is meant as a starting point.

In [1]:
import hypothesis.strategies as st
from hypothesis import given
from hypothesis.extra.datetime import dates
from functools import partial
import string
import pandas as pd

/home/khegeman/miniconda2/envs/notebooks/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


## Column Strategy
First I defined a strategy to generate dictionaries whose values are all python lists of the same length.  The generated dictionary will be passed to pandas to create the dataframe.

In [2]:
#return a dictionary whose entries all contain random integers in lists of length N. 
@st.cacheable
@st.defines_strategy
def column_st(n):
    return st.fixed_dictionaries({
                           'A':st.lists(st.integers(), min_size=n, max_size=n),
                           'B':st.lists(st.floats(), min_size=n, max_size=n),
                           })




## Timeseries Strategy
A second strategy uses the column strategy to create the dataframe with a date index.  The third parameter to build is a strategy the chooses the number of rows for the dataframe in this case between 1 and 10 rows.  Then it passes the number of rows to use the column strategy.  The lambda takes a random date to use as a starting date and the generated dictionary and constructs a dataframe.

In [3]:
@st.cacheable
@st.defines_strategy
def timeseries(column_strategy,min_year=2011, max_year=2020):
    return st.builds(        
        lambda x,y: pd.DataFrame(y).set_index(pd.date_range(x,periods=len(y.values()[0]))),
        dates(min_year=min_year, max_year=max_year),
        st.integers(min_value=1,max_value=10).flatmap(column_strategy))


In [4]:
timeseries(column_st).example()

,A,B
2016-11-28,315037,-3.905226e+18
2016-11-29,0,3.402823e+38
2016-11-30,-337013254107,1.371757e+35


## Generate Random Columns
This is just a second strategy which generates columns with random names

In [5]:


#return a dictionary with n columns, with random labels
@st.cacheable
@st.defines_strategy
def random_column_st(n,columns):
    return st.dictionaries(st.text(string.ascii_uppercase,min_size=3,max_size=6), 
                           values=st.lists(st.integers(min_value=0,max_value=100), min_size=n, max_size=n),
                           min_size=columns,max_size=columns)


In [6]:
timeseries(partial(random_column_st,columns=5)).example()

,FAAS,ITEAGK,LDY,WSBDJD,WWDFUA
2017-04-05,47,55,60,50,90
2017-04-06,7,77,67,59,99
2017-04-07,90,21,88,100,66
2017-04-08,61,21,31,12,12
2017-04-09,86,37,2,35,93
2017-04-10,23,2,88,56,22
2017-04-11,85,52,98,38,5
